In [1]:
import os
import glob
import shutil
from shutil import copyfile
import pandas as pd
import numpy as np
from pylab import imread
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from os import listdir
import random


In [6]:
trait = 'aggressive_new'
trait_abr = 'aggressive'
pair_level = 'mid-high'
gt_img_dir = '/static/images/gt_pairwise/'
prep_data_dir = '../../../preparation_data/amt_modifae_pairwise/'

n_unique_num = 80
n_rep_num = 10
n_gt_num = 10
low_post_fix = '_-1.png'
mid_post_fix = '_0.0.png'
high_post_fix = '_1.png'

cur_im_dir = '../images/modifae_pairwise/' + trait + '/'
git_im_dir = '/static/images/modifae_pairwise/' + trait + '/'

cur_dir_all_file_lst = os.listdir(cur_im_dir)
print('total file num in {} is {}'.format(cur_im_dir, len(cur_dir_all_file_lst)))
cur_dir_all_uniq_lst = set([i.split('_')[0] for i in cur_dir_all_file_lst])
print('total different num of id = {}'.format(len(cur_dir_all_uniq_lst)))

total file num in ../images/modifae_pairwise/aggressive_new/ is 300
total different num of id = 100


In [7]:
select_file_lst = list(cur_dir_all_uniq_lst)[:n_unique_num]
print('keep num = {}'.format(len(select_file_lst)))
im1_post_fix, im2_post_fix = mid_post_fix, high_post_fix
gt_im1_lst = [gt_img_dir + trait_abr + '-mid-' + str(i) + '.png' for i in range(0, n_gt_num)]
gt_im2_lst = [gt_img_dir + trait_abr + '-high-' + str(i) + '.png' for i in range(0, n_gt_num)]
im1_lst = [git_im_dir + i + im1_post_fix for i in select_file_lst]
im2_lst = [git_im_dir + i + im2_post_fix for i in select_file_lst]

stim_df = pd.DataFrame({'im1': im1_lst, 
                       'im2': im2_lst, 
                       'task_type': 'modifae', 
                       'repeat': 0, 
                       'low_first':1,
                      'pair_ind': range(0, n_unique_num)})
print('unique ele = {}'.format(stim_df['im1'].nunique()))

keep num = 80
unique ele = 80


In [10]:
stim_repeat = stim_df.sample(n_rep_num)
stim_repeat['repeat'] = 1

In [12]:
stim_repeat['im1'].nunique()

10

In [14]:
gt_df = pd.DataFrame({'im1': gt_im1_lst,
                      'im2': gt_im2_lst,
                      'task_type': 'gt',
                      'repeat': 0,
                      'low_first': 1,
                      'pair_ind': range(n_unique_num, n_unique_num+n_gt_num)})

In [15]:
gt_df['im1'].nunique()

10

In [16]:
merged_df = pd.concat([stim_df, stim_repeat, gt_df], ignore_index=True)

In [17]:
merged_df.shape

(100, 6)

In [18]:
merged_df['im1'].nunique()

90

In [29]:
df = merged_df.sample(frac=1, random_state=1)
df['im1'].nunique()

90

In [30]:
rand_lst = [random.randint(0, 1) for i in range(0, len(df))]
for ind, row in df.iterrows():
    im1_name = row['im1']
    im2_name = row['im2']

    if rand_lst[ind] == 0:
        df['im1'].loc[ind] = im2_name
        df['im2'].loc[ind] = im1_name
        df['low_first'].loc[ind] = 0

In [31]:
df[df['task_type']=='gt']

,im1,im2,low_first,pair_ind,repeat,task_type
93,/static/images/gt_pairwise/aggressive-mid-3.png,/static/images/gt_pairwise/aggressive-high-3.png,1,83,0,gt
92,/static/images/gt_pairwise/aggressive-high-2.png,/static/images/gt_pairwise/aggressive-mid-2.png,0,82,0,gt
97,/static/images/gt_pairwise/aggressive-high-7.png,/static/images/gt_pairwise/aggressive-mid-7.png,0,87,0,gt
94,/static/images/gt_pairwise/aggressive-mid-4.png,/static/images/gt_pairwise/aggressive-high-4.png,1,84,0,gt
99,/static/images/gt_pairwise/aggressive-mid-9.png,/static/images/gt_pairwise/aggressive-high-9.png,1,89,0,gt
95,/static/images/gt_pairwise/aggressive-mid-5.png,/static/images/gt_pairwise/aggressive-high-5.png,1,85,0,gt
91,/static/images/gt_pairwise/aggressive-mid-1.png,/static/images/gt_pairwise/aggressive-high-1.png,1,81,0,gt
98,/static/images/gt_pairwise/aggressive-high-8.png,/static/images/gt_pairwise/aggressive-mid-8.png,0,88,0,gt
90,/static/images/gt_pairwise/aggressive-mid-0.png,/static/images/gt_pairwise/aggressive-high-0.png,1,80,0,gt
96,/static/images/gt_pairwise/aggressive-high-6.png,/static/images/gt_pairwise/aggressive-mid-6.png,0,86,0,gt


In [32]:
save_csv_name = prep_data_dir + trait + '-' + pair_level + '-stim-lst.csv'
df.to_csv(save_csv_name, index=False)

# write the df into txt format. list of lists.
test_lst = []
for ind, row in df.iterrows():
    test_lst.append(row.values.tolist())

save_txt_name = prep_data_dir + trait + '-' + pair_level + '.txt'

with open(save_txt_name, 'w') as f:
    f.write("var stims = [\n")
    for cur_line in test_lst:
        f.write("{},\n".format(cur_line))
    f.write('];\n')